In [4]:
import numpy as np
data = np.load('C:\\Users\\zihan\\Desktop\\Files\\Research\\bryankelly_without_missing.npz')
processed_data = data['processed_data']
T, N, M = processed_data.shape
L= 4
D= 10
# The dimension of returns: T*N
chara_data = processed_data[:, :, [8, 17, 20, 33, 37]]
# chara_data= processed_data[:,:,2:]
# 20: seem to be accrual

In [5]:
fama_french_data= np.load('C:\\Users\\zihan\\Desktop\\Files\\Research\\riskfreerate.npz')
fama_french_data= fama_french_data['fama_french_data']
r_f= fama_french_data[:,4]
r_f= r_f/100
date= fama_french_data[:,0]
date_start= np.asscalar(np.argwhere(date==198406))
date_end= np.asscalar(np.argwhere(date==201405))
returns = processed_data[:, :, 1]
returns= returns- (r_f[date_start:date_end+1:,np.newaxis])
accural= chara_data[:,:,1]
size= chara_data[:,:,2]
T,N= returns.shape
M = 5
portfolio = np.zeros((T-1,25))
for i in range(T-1):
# for i in range(0,15):
    return_t= returns[i+1,:]
    basis= np.zeros((N, M*M))
    for j in range(M):
        for k in range(M):
            quantile_accural_lower= np.percentile(accural[i,:], int(100.0/M*(j)))
            quantile_accural_upper= np.percentile(accural[i,:], int(100.0/M*(j+1)))
            quantile_size_lower= np.percentile(size[i,:], int(100.0/M*(k)))
            quantile_size_upper = np.percentile(size[i,:], int(100.0/M*(k+1)))
            index= (accural[i,:]>= quantile_accural_lower)& (accural[i,:]<= quantile_accural_upper)& (size[i,:]>= quantile_size_lower)& (size[i,:]<= quantile_size_upper)
            portfolio[i, j*M+k]= np.mean(return_t[index])
# portfolio[np.isnan(portfolio)]= 0
    #         basis[:,j*M+k]= (accural[i,:]>= quantile_accural_lower) & (accural[i,:]<= quantile_accural_upper)& (size[i,:]>= quantile_size_lower)& (size[i,:]<= quantile_size_upper)
    # portfolio[i,:]= return_t.dot(basis)

In [6]:
import scipy.linalg as la
M= portfolio.shape[1]
K = 3
window= 120
optimal_return = np.zeros((T-1-window))
gamma = 100
for t in range(T-1-window):
# for t in range(0,1):
    if t % 20 == 0:
        print('Have gone through iteration %d' % (t))
    portfolio_t = portfolio[t:(t + window), :]
    covmat_t = portfolio_t.T.dot(portfolio_t) 
    portfolio_mean_t = np.sum(portfolio_t, axis=0, keepdims= True)
    covmat_t = covmat_t + portfolio_mean_t.T.dot(portfolio_mean_t) / window * gamma
    variance_t, loading_t= la.eigh(covmat_t, eigvals=(M-K, M-1))
    Factor_t= portfolio_t.dot(loading_t)
    mu= np.mean(Factor_t, axis= 0)
    covmat_factor= np.cov(Factor_t.T)
    weight= np.linalg.inv(covmat_factor).dot(mu)
    optimal_return[t] = (portfolio[t + window, :].dot(loading_t)).dot(weight)
mean_optimal= np.mean(optimal_return)
cov_optimal= np.cov(optimal_return)
print (abs(mean_optimal)/np.sqrt(cov_optimal))

Have gone through iteration 0
Have gone through iteration 20
Have gone through iteration 40
Have gone through iteration 60
Have gone through iteration 80
Have gone through iteration 100
Have gone through iteration 120
Have gone through iteration 140
Have gone through iteration 160
Have gone through iteration 180
Have gone through iteration 200
Have gone through iteration 220
0.24168898616


In [11]:
size_accrual_fama= np.load('C:\\Users\\zihan\\Desktop\\Files\\Research\\size_accrual_fama.npz')
size_accrual_fama= size_accrual_fama['size_accrual_fama']
portfolio_date= size_accrual_fama[:,0]
portfolio_date_start= np.asscalar(np.argwhere(portfolio_date==198406))
portfolio_date_end= np.asscalar(np.argwhere(portfolio_date==201405))
fama_portfolio= size_accrual_fama[portfolio_date_start:portfolio_date_end+1,1:26]/100
portfolio= fama_portfolio
portfolio= portfolio- (r_f[date_start:date_end+1:,np.newaxis])
import scipy.linalg as la
M= portfolio.shape[1]
K = 3
window= 120
optimal_return = np.zeros((T-1-window))
gamma = 100
for t in range(T-1-window):
# for t in range(0,1):
    if t % 20 == 0:
        print('Have gone through iteration %d' % (t))
    portfolio_t = portfolio[t:(t + window), :]
    covmat_t = portfolio_t.T.dot(portfolio_t) 
    portfolio_mean_t = np.sum(portfolio_t, axis=0, keepdims= True)
    covmat_t = covmat_t + portfolio_mean_t.T.dot(portfolio_mean_t) / window * gamma
    variance_t, loading_t= la.eigh(covmat_t, eigvals=(M-K, M-1))
    Factor_t= portfolio_t.dot(loading_t)
    mu= np.mean(Factor_t, axis= 0)
    covmat_factor= np.cov(Factor_t.T)
    weight= np.linalg.inv(covmat_factor).dot(mu)
    optimal_return[t] = (portfolio[t + window, :].dot(loading_t)).dot(weight)
mean_optimal= np.mean(optimal_return)
cov_optimal= np.cov(optimal_return)
print (abs(mean_optimal)/np.sqrt(cov_optimal))

Have gone through iteration 0
Have gone through iteration 20
Have gone through iteration 40
Have gone through iteration 60
Have gone through iteration 80
Have gone through iteration 100
Have gone through iteration 120
Have gone through iteration 140
Have gone through iteration 160
Have gone through iteration 180
Have gone through iteration 200
Have gone through iteration 220
0.169675941224


In [8]:
gamma= 100
# portfolio_demean = portfolio - np.mean(portfolio, axis=0)
# Dimension of portfolio: T*M
covmat = portfolio.T.dot(portfolio) 
portfolio_mean= np.sum(portfolio, axis= 0, keepdims=True)
covmat= covmat+ portfolio_mean.T.dot(portfolio_mean)/(T-1)*gamma
variance, loading = np.linalg.eig(covmat)
output= []
for K in range(1, 20):
    loading_chosen = loading[:, :K]
    Factor = portfolio.dot(loading_chosen)
    mu = np.mean(Factor, axis=0)
    covmat_factor = np.cov(Factor.T)
    if K==1:
        print (K)
        output.append (abs(mu)*covmat_factor**(-1/2))
    else:
        output.append(np.sqrt(np.dot(np.dot(mu, np.linalg.inv(covmat_factor)), mu.T)))
twodecimals = ["&%.2f" % np.real(v) for v in output]
print (''.join(twodecimals))

1
&0.15&0.18&0.30&0.34&0.34&0.35&0.35&0.35&0.38&0.38&0.38&0.39&0.40&0.40&0.40&0.40&0.40&0.40&0.40


In [12]:
# Get the pricing error
import scipy.linalg as la
K = 4
window= 120
gamma = 100
T, M = portfolio.shape
pricing_error= np.zeros((T-window,M ))
ones_T= np.ones((window, 1))
for t in range(T-window):
# for t in range(0,1):
    if t % 20 == 0:
        print('Have gone through iteration %d' % (t))
    portfolio_t = portfolio[t:(t + window), :]
    # covmat_t = portfolio_t.T.dot(portfolio_t) 
    # portfolio_mean_t = np.sum(portfolio_t, axis=0, keepdims= True)
    # # print (portfolio_mean_t.shape)
    # covmat_t = covmat_t + portfolio_mean_t.T.dot(portfolio_mean_t) / window * gamma
    covmat_t= portfolio_t.dot(portfolio_t.T).dot(gamma/T* ones_T.dot(ones_T.T))
    covmat_t= covmat_t+ portfolio_t.dot(portfolio_t.T)
    # variance_t, loading_t = np.linalg.eig(covmat_t)
    variance_t, Factor_t= la.eigh(covmat_t, eigvals=(window-K, window-1))
    loading_t= Factor_t.T.dot(portfolio_t)
    # loading_t: K*M.
    # Factor_t= portfolio_t.dot(loading_t)
    pricing_error[t,:] = (portfolio[t + window, :]- portfolio[t+window, :].dot(loading_t.T).dot(np.linalg.inv(loading_t.dot(loading_t.T))).dot(loading_t))
print (np.sqrt(np.sum(pricing_error**2)/M)/(T-window))

Have gone through iteration 0
Have gone through iteration 20
Have gone through iteration 40
Have gone through iteration 60
Have gone through iteration 80
Have gone through iteration 100
Have gone through iteration 120
Have gone through iteration 140
Have gone through iteration 160
Have gone through iteration 180
Have gone through iteration 200
Have gone through iteration 220
0.00112078553376


In [13]:
# In sample pricing error
import scipy.linalg as la
T,M= portfolio.shape
K= 5
gamma= 100
ones_T= np.ones((T,1))
# portfolio_demean = portfolio - np.mean(portfolio, axis=0)
# Dimension of portfolio: T*M
# covmat = portfolio.T.dot(portfolio) 
# portfolio_mean= np.sum(portfolio, axis= 0, keepdims=True)
# covmat= covmat+ portfolio_mean.T.dot(portfolio_mean)/(T-1)*gamma
# variance, loading = la.eigh(covmat, (M-K, M-1))
covmat= portfolio.dot(portfolio.T).dot(gamma/T* ones_T.dot(ones_T.T))
covmat= covmat+ portfolio.dot(portfolio.T)
    # variance_t, loading_t = np.linalg.eig(covmat_t)
variance, Factor= la.eigh(covmat, eigvals=(T-K, T-1))
loading= Factor.T.dot(portfolio)
pricing_error= portfolio- Factor.dot(loading)
print (np.sqrt(np.sum(pricing_error**2)/M)/(T))

0.00210350596256
